# Task 1 (6p)
Your task is to modify the custom implementation of MultiHeadAttention. This custom implementation, currently, enables each token to attent to every other token.


Your job is to change this behavior in a specific way.
Let $S$ be our input sequence of length $2 \cdot k$:
- tokens on positions $i \lt k$ should attend to prefix of $S$ of length $k$ ($S[:k]$) - every token up to position k
- tokens on positions $i \ge k$ should attend to prefix of $S$  of length $i + 1$ ($S[:i + 1]$) - every previous token and itself

(Note: You can assume the sequence length is always an even number).

In [1]:
import torch
import math
import torch.nn.functional as F
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_head):
      super().__init__()
      self.d_model = d_model
      self.num_heads = num_heads
      self.d_head = d_head

      self.W_Q = torch.nn.Linear(d_model, num_heads*d_head, bias=True)
      self.W_K = torch.nn.Linear(d_model, num_heads*d_head, bias=True)
      self.W_V = torch.nn.Linear(d_model, num_heads*d_head, bias=True)
      self.W_O = torch.nn.Linear(num_heads*d_head, d_model, bias=True)

    def forward(self, x):

      seq_len, batch_size, _ = x.shape

      Q = self.W_Q(x).reshape(seq_len, batch_size, self.num_heads, self.d_head)
      K = self.W_K(x).reshape(seq_len, batch_size, self.num_heads, self.d_head)
      V = self.W_V(x).reshape(seq_len, batch_size, self.num_heads, self.d_head)

      scaled_QK = torch.einsum("ibhd,jbhd->bhij", Q, K) / math.sqrt(self.d_head)
      # shape of scaled_QK is (batch_size, num_heads, seq_len, seq_len)
      #TODO
      k = seq_len // 2
      mask = torch.triu(torch.ones_like(scaled_QK), diagonal = 1)
      mask[:, :, :, :k] = 0
      mask = mask.masked_fill(mask == 1, float('-inf'))

      scaled_QK = scaled_QK + mask
      #ENDTODO
      weights = F.softmax(scaled_QK, -1)
      attention = torch.einsum("bhij,jbhd->ibhd", weights, V)

      result = self.W_O(attention.reshape(seq_len, batch_size,self.num_heads * self.d_head))

      return result, weights

In [2]:
# Test your solution
d_model = 2
num_heads= 3
d_head = 3
k = 3
batch_size = 10

mha = MultiHeadAttention(d_model, num_heads, d_head)
batched_x= torch.randn((2*k, batch_size, d_model))
with torch.no_grad():
  result, weights = mha(batched_x)
print("Result:", result)
print("Weights:", weights)

Result: tensor([[[-0.0738, -0.7158],
         [-0.2962, -0.8647],
         [-0.0551, -0.6896],
         [-0.0719, -0.7750],
         [-0.0245, -0.6096],
         [-0.2812, -0.8499],
         [-0.0889, -0.6623],
         [ 0.0045, -0.6231],
         [ 0.1234, -0.6266],
         [ 0.1969, -0.4433]],

        [[-0.0864, -0.7064],
         [-0.3097, -0.8725],
         [-0.0520, -0.7458],
         [-0.0635, -0.7501],
         [-0.0246, -0.6095],
         [-0.2700, -0.8494],
         [-0.0617, -0.6447],
         [ 0.0098, -0.6296],
         [ 0.0193, -0.5692],
         [ 0.2157, -0.4728]],

        [[-0.0749, -0.7412],
         [-0.2959, -0.8738],
         [-0.0695, -0.7749],
         [-0.1043, -0.6927],
         [-0.0245, -0.6094],
         [-0.2702, -0.8395],
         [-0.0772, -0.6421],
         [ 0.0099, -0.6196],
         [ 0.0804, -0.5689],
         [ 0.2238, -0.4892]],

        [[-0.0244, -0.6709],
         [-0.2381, -0.8245],
         [-0.0379, -0.6989],
         [-0.0304, -0.6932],


In [3]:
#some code to think:

k = 3
A = torch.ones(2*k,2*k)

mask = torch.triu(torch.ones_like(A), diagonal = 1)
print(mask)
mask[:,:k] = 0
print(mask)
mask = mask.masked_fill(mask == 1, float('-inf'))


print(mask)
print(A+mask)
print(torch.nn.functional.softmax(input = (A+mask), dim = -1))


tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0.]])
tensor([[1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000],
        [0.3333,